In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sqlalchemy import create_engine


df = pd.read_csv(r"C:\Users\Thennarasu\OneDrive\Documents\dataset\ICRISAT-District Level Data - ICRISAT-District Level Data.csv")
df.isna().sum()
def clean_column(col):
    col = col.strip()  # remove leading/trailing whitespace
    col = col.lower()  # lowercase everything
    col = re.sub(r"\(.*?\)", "", col)  # remove anything in parentheses like (1000 ha)
    col = col.replace(" ", "_")  # replace spaces with underscores
    col = re.sub(r"[^a-z0-9_]", "", col)  # remove any remaining special characters
    return col.rstrip('_')
df.columns = [clean_column(col) for col in df.columns]
print(df.columns)

username = 'postgres'
password = 'thennarasu'
host = 'localhost'
port = '5432'
database = 'secure'
schema = 'public'  # Example: 'public'

# Create connection string and engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

# Insert data into PostgreSQL
df.to_sql(name='agri', con=engine, schema=schema, if_exists='replace', index=False) 

v = 'successfully'

print(f"welcom {v} inseted")






In [ ]:
## 	Top 7 RICE PRODUCTION State Data ##

top_rice_states = df.groupby('state_name')['rice_production'].sum().sort_values(ascending=False).head(7)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_rice_states.values, y=top_rice_states.index, palette='viridis')
plt.title('Top 7 Rice-Producing States in India')
plt.xlabel('Total Rice Production (1000 tons)')
plt.ylabel('State')
plt.tight_layout()
plt.show()


## Wheat Production Percentage Share of Top 5 States ##

# Pie Chart
top_wheat_states = df.groupby('state_name')['wheat_production'].sum().sort_values(ascending=False).head(5)
plt.figure(figsize=(6, 6))
plt.pie(top_wheat_states.values,
        labels=top_wheat_states.index,
        autopct='%1.1f%%',  # Shows percentage
        startangle=140,
        colors=sns.color_palette('Blues', n_colors=5))

plt.title('Wheat Production Percentage Share of Top 5 States')
plt.axis('equal')  # Equal aspect ratio ensures pie is a circle
plt.tight_layout()
plt.show()

## Top 10 Oilseed Producing States in India ##

# Group by state and sum total oilseeds production
oilseed_by_state = df.groupby('state_name')['oilseeds_production'].sum().sort_values(ascending=False)

# Select top 10 states
top_oilseed_states = oilseed_by_state.head(10)

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x=top_oilseed_states.index, y=top_oilseed_states.values, palette='YlOrBr')

# Titles and labels
plt.title('Top 10 Oilseed Producing States in India', fontsize=16)
plt.xlabel('State', fontsize=12)
plt.ylabel('Total Oilseeds Production (in 1000 tons)', fontsize=12)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


## 	Top 7 SUNFLOWER PRODUCTION  State ##

top_sunflower_states = df.groupby('state_name')['sunflower_production'] \
                         .sum() \
                         .sort_values(ascending=False) \
                         .head(7)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x=top_sunflower_states.values, y=top_sunflower_states.index, palette="YlOrBr")
plt.title("Top 7 Sunflower Producing States in India")
plt.xlabel("Total Sunflower Production (1000 Tons)")
plt.ylabel("State")
plt.tight_layout()
plt.show()



## India's Sugarcane Production Over the Last 50 Years ##

sugarcane_trend = df.groupby('year')['sugarcane_production'].sum().reset_index()

# Line plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=sugarcane_trend, x='year', y='sugarcane_production', marker='o', color='green')

plt.title("India's Sugarcane Production Over the Last 50 Years")
plt.xlabel("Year")
plt.ylabel("Total Sugarcane Production (1000 tons)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

## Rice vs Wheat Production in India (Last 50 Years) ##

rice_wheat_trend = df.groupby('year')[['rice_production', 'wheat_production']].sum().reset_index()

# Plotting
plt.figure(figsize=(12, 6))
sns.lineplot(data=rice_wheat_trend, x='year', y='rice_production', label='Rice', marker='o', color='green')
sns.lineplot(data=rice_wheat_trend, x='year', y='wheat_production', label='Wheat', marker='s', color='orange')

plt.title('Rice vs Wheat Production in India (Last 50 Years)')
plt.xlabel('Year')
plt.ylabel('Production (1000 tons)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



## 	Rice Production By West Bengal Districts ##

# Filter for West Bengal
wb_rice = df[df['state_name'].str.strip().str.lower() == 'west bengal']

# Group by district and sum rice production
district_rice = wb_rice.groupby('dist_name')['rice_production'] \
                       .sum() \
                       .sort_values(ascending=False)

# Plotting the top 10 districts
plt.figure(figsize=(12, 6))
sns.barplot(x=district_rice.head(10).values, y=district_rice.head(10).index, palette="Greens_d")
plt.xlabel("Total Rice Production (1000 Tons)")
plt.ylabel("District")
plt.title("Top 10 Rice Producing Districts in West Bengal")
plt.tight_layout()
plt.show()

## Top 10 Wheat Production Years From UP ##

up_wheat = df[df['state_name'].str.strip().str.lower() == 'uttar pradesh']
top_years = up_wheat.groupby('year')['wheat_production'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_years.index, y=top_years.values, palette="Blues_d")
plt.title("Top 10 Years of Wheat Production in Uttar Pradesh")
plt.xlabel("Year")
plt.ylabel("Wheat Production (1000 Tons)")
plt.tight_layout()
plt.show()

## Millet Production (Last 50y) ##

df['TOTAL MILLET PRODUCTION'] = (
    df['pearl_millet_production'].fillna(0) +
    df['finger_millet_production'].fillna(0) 
)
millet_trend = df.groupby('year')['TOTAL MILLET PRODUCTION'].sum().sort_index()
millet_trend_last_50 = millet_trend.tail(50)

plt.figure(figsize=(12,6))
plt.plot(millet_trend_last_50.index, millet_trend_last_50.values, marker='o', color='olive')
plt.title("Millet Production in India (Last 50 Years)")
plt.xlabel("Year")
plt.ylabel("Total Millet Production (1000 Tons)")
plt.grid(True)
plt.tight_layout()
plt.show()


## Soybean Production and Yield Efficiency - Top 5 States ##


# Group by state
soybean_df = df.groupby('state_name')[['soyabean_production', 'soyabean_yield']].sum()
top_soy_states = soybean_df.sort_values(by='soyabean_production', ascending=False).head(5)

# Dual axis bar plot
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.bar(top_soy_states.index, top_soy_states['soyabean_production'], color=color)
ax1.set_ylabel('Soybean Production (1000 tons)', color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # Second y-axis
color = 'tab:green'
ax2.plot(top_soy_states.index, top_soy_states['soyabean_yield'], color=color, marker='o')
ax2.set_ylabel('Soybean Yield (Kg/ha)', color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Soybean Production and Yield Efficiency - Top 5 States')
plt.tight_layout()
plt.show()


## Sorghum Production (Kharif and Rabi) by Top 10 States ##

sorghum_cols = ['kharif_sorghum_production', 'rabi_sorghum_production']

# Group by state
sorghum_by_state = df.groupby('state_name')[sorghum_cols].sum().sort_values(by='kharif_sorghum_production', ascending=False)

# Stacked bar chart
sorghum_by_state.head(10).plot(kind='bar', stacked=True, figsize=(12, 6), colormap='Accent')
plt.title('Sorghum Production (Kharif and Rabi) by Top 10 States')
plt.xlabel('State')
plt.ylabel('Production (1000 tons)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



## Top 7 States for Groundnut Production ##

top_groundnut_states = df.groupby('state_name')['groundnut_production'] \
                         .sum() \
                         .sort_values(ascending=False) \
                         .head(7)

# Plot the result
plt.figure(figsize=(10, 6))
sns.barplot(x=top_groundnut_states.values, y=top_groundnut_states.index, palette="Oranges_d")
plt.title("Top 7 Groundnut Producing States in India")
plt.xlabel("Total Groundnut Production (1000 Tons)")
plt.ylabel("State")
plt.tight_layout()
plt.show()

## Rice vs. Wheat Yield Across States ##

# Average yield by state
yield_df = df.groupby('state_name')[['rice_yield', 'wheat_yield']].mean()

# Plot
yield_df = yield_df.sort_values('rice_yield', ascending=False)

yield_df.plot(kind='bar', figsize=(14, 6), colormap='Set2')
plt.title('Average Rice vs Wheat Yield Across States')
plt.xlabel('State')
plt.ylabel('Yield (Kg/ha)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(16, 5))

for i, crop in enumerate(['rice', 'wheat', 'maize']):
    plt.subplot(1, 3, i + 1)
    sns.regplot(x=f'{crop}_area', y=f'{crop}_production', data=df, scatter_kws={'alpha':0.5})
    plt.title(f'{crop.capitalize()} Area vs Production')
    plt.xlabel('Area (1000 ha)')
    plt.ylabel('Production (1000 tons)')

plt.tight_layout()
plt.show()


